# 2D Fourier Transform

In [1]:
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
images = [ cv2.imread("./data/00{}.jpg".format(i), cv2.IMREAD_GRAYSCALE) for i in range(1, 7)] 
import time

## Matrix Multiplication

In [40]:
class Matrix:
    def __init__(self, data):
        self.data = data
    
    def __repr__(self):
        return self.data.__repr__()
    
    def __getitem__(self, i):
        return self.data[i]
    
    def __truediv__(self, denom):
        div_mat = self.zeros()
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                div_mat[i][j] = self[i][j] / denom 
        return Matrix(div_mat)
    
    def __len__(self):
        return len(self.data), len(self.data[0])
    @property 
    def zeros(self):
        return [[0 for _ in range(self.shape[1])] for _ in range(self.shape[0])]
    
    @property
    def shape(self):
        return (len(self.data), len(self.data[0]))
    
    def round(self, n=8):
        rounded = self.zeros
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                re = self.data[i][j].real
                im = self.data[i][j].imag
                if im == 0:
                    rounded[i][j] = round(re, n)
                else:
                    rounded[i][j] = round(re, n) + round(im, n) * 1j
        return Matrix(rounded)
        
    
    def matmul(self, B):
        result = [[0 for _ in range(B.shape[1])] for _ in range(self.shape[0])]
        for i in range(self.shape[0]):
            for j in range(B.shape[1]):
                sum = 0
                for k in range(self.shape[1]):
                    sum += self[i][k] * B[k][j]
                result[i][j] = sum 
        return Matrix(result)
    
    @property
    def transpose(self):
        result = [[0 for _ in range(self.shape[0])] for _ in range(self.shape[1])]
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                result[j][i] = self[i][j]
                
    @property
    def exp_mu(self, reverse = False):
        M = self.shape[0]
        result = [[0 for _ in range(M)] for _ in range(M)]
        for u in range(M):
            for m in range(M):
                if reverse:
                    result[m][u] = np.exp(2j* np.pi * m * u / M )
                else: 
                    result[m][u] = np.exp(-2j* np.pi * m * u / M )
        return Matrix(result)
    @property
    def exp_nu(self, reverse = True):
        N = self.shape[1]
        result = [[0 for _ in range(N)] for _ in range(N)]
        for v in range(N):
            for n in range(N):
                if reverse:
                    result[n][v] = np.exp(2j* np.pi * n * v / N )
                else:
                    result[n][v] = np.exp(-2j* np.pi * n * v / N )
                
        return Matrix(result)
    
    def re_im(self):
        re = self.zeros
        im = self.zeros
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                re[i][j] = self[i][j].real
                im[i][j] = self[i][j].imag
        return Matrix(re), Matrix(im) 
    
    @property
    def magnitude(self):
        mag = self.zeros
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                mag[i][j] = abs(self[i][j])
        return Matrix(mag)
    
    def shift(self):
        return Matrix(np.fft.fftshift(self.data))
    
    def show(self):
        mat = self.shift().magnitude.round(0)
        plt.imshow(mat.data, cmap='gray')
            
A = Matrix([[1, 2, 3], [4, 5, 6]])
B = Matrix([[1, 2], [3, 4], [5, 6]])

## 2D Fourier Transform

In [52]:
def fft2(img: Matrix): 
    M, N = img.shape 
    print("processing... might take a while")
    fourier = img.exp_mu.matmul(img).matmul(img.exp_nu) 
    fourier.round()
    print("process ended")
    return fourier

def inverse_fft2(img: Matrix):
    M, N = img.shape 
    print("processing... might take a while")
    inv_fourier = img.exp_nu(reversed = True).matmul(img).matmul(img.exp_mu(reversed = True))
    inv_fourier.round()
    print("process ended")
    return inv_fourier

In [53]:
img = Matrix(images[0])
fft_img = fft2(img)

processing... might take a while


## Azimuthal Averaging

In [43]:
def azimuthal_averaging(img: Matrix):
    h, w = img.shape 
    center = (h // 2, w // 2) 
    max_radius = np.sqrt(center[0] ** 2 + center[1] ** 2) 
    if max_radius != int(max_radius):
        max_radius += 1
    max_radius = int(max_radius)
    cum_sum_freq = [0 for _ in range(max_radius)]
    
    for i in range(h):
        for j in range(w):
            radius = np.sqrt((i - center[0]) ** 2 + (j - center[1]) ** 2)
            cum_sum_freq[int(radius)] += img[i][j]
            
    cum_sum_freq.sort(reverse= True)
    return cum_sum_freq
    

In [44]:
fft_mag = fft_img.magnitude.round(0)
az = azimuthal_averaging(fft_mag)

In [50]:
inv_fft = inverse_fft2(fft_img)

processing... might take a while


TypeError: Matrix.matmul() got an unexpected keyword argument 'reverse'